In [1]:
import pandas as pd

# Load raw data
data = pd.read_excel("data/Analytics and Forecasting - Calls Data.xlsx", sheet_name="Data")

# Basic cleaning
data['DAY'] = pd.to_datetime(data['DAY'])
data['DATE'] = data['DAY'].dt.date
data['MONTH'] = data['DAY'].dt.month
data['MONTH_NAME'] = data['DAY'].dt.strftime('%B')
data['DAY_OF_WEEK'] = data['DAY'].dt.day_name()
data['QUARTER'] = data['DAY'].dt.quarter
data['TOTAL_CALLS'] = data['CALLS HANDLED'] + data['CALLS MISSED']

data.head()


,DAY,CALLS HANDLED,CALLS MISSED,QUEUE,VENDOR_COMPANY,SATISFIED,CALLS WITHIN SLA,IS OUTAGE ?,DATE,MONTH,MONTH_NAME,DAY_OF_WEEK,QUARTER,TOTAL_CALLS
0,2024-01-01,1060.5,73.5,Customer Queue,Vendor A,460,986.986029,True,2024-01-01,1,January,Monday,1,1134.0
1,2024-01-01,2151.0,73.5,Customer Queue,Vendor B,235,1789.695254,False,2024-01-01,1,January,Monday,1,2224.5
2,2024-01-01,6667.5,322.5,Customer Queue - Vendor B,Vendor B,4297,6214.145123,False,2024-01-01,1,January,Monday,1,6990.0
3,2024-01-01,10255.5,63.0,Customer Queue - Vendor A,Vendor A,4099,9004.283723,True,2024-01-01,1,January,Monday,1,10318.5
4,2024-01-02,718.5,13.5,Customer Queue,Vendor A,125,650.914783,True,2024-01-02,1,January,Tuesday,1,732.0


In [2]:
# Overall totals
total_calls = data['TOTAL_CALLS'].sum()
total_handled = data['CALLS HANDLED'].sum()

# Split queues
shared = data[data['QUEUE'] == 'Customer Queue']
vend_a_q = data[data['QUEUE'] == 'Customer Queue - Vendor A']
vend_b_q = data[data['QUEUE'] == 'Customer Queue - Vendor B']

# Actual total missed (shared counted once)
shared_missed_actual = shared['CALLS MISSED'].sum()
vend_a_missed_ded = vend_a_q['CALLS MISSED'].sum()
vend_b_missed_ded = vend_b_q['CALLS MISSED'].sum()
actual_total_missed = shared_missed_actual + vend_a_missed_ded + vend_b_missed_ded

miss_rate = actual_total_missed / total_calls * 100

total_calls, total_handled, actual_total_missed, miss_rate


(4804644.0, 4645473.0, 159171.0, 3.3128573105520407)

In [3]:
# Vendor-level volumes (attributed)
vendor_vol = (
    data.groupby('VENDOR_COMPANY')
        .agg(calls_handled=('CALLS HANDLED', 'sum'))
)

vendor_vol['percent_of_handled'] = vendor_vol['calls_handled'] / vendor_vol['calls_handled'].sum() * 100
vendor_vol


,calls_handled,percent_of_handled
VENDOR_COMPANY,,
Vendor A,2725531.5,58.670699
Vendor B,1919941.5,41.329301


In [4]:
# Calls picked up (handled) in each queue
queue_handled = (
    data.groupby('QUEUE')['CALLS HANDLED']
        .sum()
        .rename("calls_handled")
        .reset_index()
)

# Calls missed in each queue (raw, as stored)
queue_missed_raw = (
    data.groupby('QUEUE')['CALLS MISSED']
        .sum()
        .rename("calls_missed_raw")
        .reset_index()
)

queue_handled, queue_missed_raw


(                       QUEUE  calls_handled
 0             Customer Queue       782022.0
 1  Customer Queue - Vendor A      2305251.0
 2  Customer Queue - Vendor B      1558200.0,
                        QUEUE  calls_missed_raw
 0             Customer Queue           31575.0
 1  Customer Queue - Vendor A           85944.0
 2  Customer Queue - Vendor B           41652.0)

In [5]:
# Missed calls by vendor (attributed)

shared_a = shared[shared['VENDOR_COMPANY'] == 'Vendor A']['CALLS MISSED'].sum()
shared_b = shared[shared['VENDOR_COMPANY'] == 'Vendor B']['CALLS MISSED'].sum()

vend_a_missed_attrib = shared_a + vend_a_missed_ded
vend_b_missed_attrib = shared_b + vend_b_missed_ded

missed_by_vendor = pd.DataFrame({
    'vendor': ['Vendor A', 'Vendor B'],
    'missed_calls_attributed': [vend_a_missed_attrib, vend_b_missed_attrib]
})

missed_by_vendor


,vendor,missed_calls_attributed
0,Vendor A,101731.5
1,Vendor B,57439.5


In [6]:
# Overall satisfaction
total_satisfied = data['SATISFIED'].sum()
overall_satisfaction_rate = total_satisfied / total_handled * 100

overall_satisfaction_rate


38.23401836583702

In [7]:
vendor_sat = (
    data.groupby('VENDOR_COMPANY')
        .agg(
            satisfied=('SATISFIED', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum')
        )
)

vendor_sat['satisfaction_rate'] = vendor_sat['satisfied'] / vendor_sat['calls_handled'] * 100
vendor_sat


,satisfied,calls_handled,satisfaction_rate
VENDOR_COMPANY,,,
Vendor A,1017772,2725531.5,37.342148
Vendor B,758379,1919941.5,39.500110


In [8]:
# Overall SLA
total_calls_within_sla = data['CALLS WITHIN SLA '].sum()
overall_sla_rate = total_calls_within_sla / total_calls * 100

overall_sla_rate


87.99766620968764

In [9]:
vendor_sla = (
    data.groupby('VENDOR_COMPANY')
        .agg(
            calls_within_sla=('CALLS WITHIN SLA ', 'sum'),
            total_calls=('TOTAL_CALLS', 'sum')
        )
)

vendor_sla['sla_rate'] = vendor_sla['calls_within_sla'] / vendor_sla['total_calls'] * 100
vendor_sla


,calls_within_sla,total_calls,sla_rate
VENDOR_COMPANY,,,
Vendor A,2.558861e+06,2827228.5,90.507739
Vendor B,1.669114e+06,1977415.5,84.408866


In [11]:
# SLA on outage vs non-outage days

outage_group = (
    data.groupby('IS OUTAGE ?')
        .agg(
            calls_within_sla=('CALLS WITHIN SLA ', 'sum'),
            total_calls=('TOTAL_CALLS', 'sum')
        )
        .reset_index()
)

# Correct: divide by the total_calls column only
outage_group['sla_rate'] = outage_group['calls_within_sla'] / outage_group['total_calls'] * 100

outage_group


,IS OUTAGE ?,calls_within_sla,total_calls,sla_rate
0,False,3.586321e+06,4069143.0,88.134542
1,True,6.416540e+05,735501.0,87.240403


In [12]:
sla_outage = outage_group.loc[outage_group['IS OUTAGE ?'] == True, 'sla_rate'].iloc[0]
sla_non_outage = outage_group.loc[outage_group['IS OUTAGE ?'] == False, 'sla_rate'].iloc[0]
impact = sla_non_outage - sla_outage
sla_outage, sla_non_outage, impact


(87.24040255879936, 88.13454224783422, 0.8941396890348585)

In [13]:
# Day-of-week performance patterns

dow_perf = (
    data.groupby('DAY_OF_WEEK')
        .agg(
            total_calls=('TOTAL_CALLS', 'sum'),
            calls_handled=('CALLS HANDLED', 'sum'),
            calls_missed_raw=('CALLS MISSED', 'sum'),
            satisfied=('SATISFIED', 'sum'),
            calls_within_sla=('CALLS WITHIN SLA ', 'sum')
        )
        .reset_index()
)

dow_perf['miss_rate'] = dow_perf['calls_missed_raw'] / dow_perf['total_calls'] * 100
dow_perf['satisfaction_rate'] = dow_perf['satisfied'] / dow_perf['calls_handled'] * 100
dow_perf['sla_rate'] = dow_perf['calls_within_sla'] / dow_perf['total_calls'] * 100

dow_perf


,DAY_OF_WEEK,total_calls,calls_handled,calls_missed_raw,satisfied,calls_within_sla,miss_rate,satisfaction_rate,sla_rate
0,Friday,721254.0,698647.5,22606.5,255347,633838.523843,3.134333,36.548760,87.880071
1,Monday,684493.5,662386.5,22107.0,272608,603161.380039,3.229687,41.155428,88.117912
2,Saturday,728715.0,702568.5,26146.5,279605,642342.228578,3.588028,39.797543,88.147249
3,Sunday,730203.0,705399.0,24804.0,249312,642788.596102,3.396864,35.343401,88.028753
4,Thursday,677407.5,653925.0,23482.5,257214,597093.276249,3.466525,39.333869,88.143883
5,Tuesday,628299.0,606333.0,21966.0,223558,552041.202195,3.496106,36.870499,87.862817
6,Wednesday,634272.0,616213.5,18058.5,238507,556709.382678,2.847122,38.705254,87.771395


In [14]:
day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dow_perf['DAY_OF_WEEK'] = pd.Categorical(dow_perf['DAY_OF_WEEK'], categories=day_order, ordered=True)
dow_perf = dow_perf.sort_values('DAY_OF_WEEK')
dow_perf


,DAY_OF_WEEK,total_calls,calls_handled,calls_missed_raw,satisfied,calls_within_sla,miss_rate,satisfaction_rate,sla_rate
1,Monday,684493.5,662386.5,22107.0,272608,603161.380039,3.229687,41.155428,88.117912
5,Tuesday,628299.0,606333.0,21966.0,223558,552041.202195,3.496106,36.870499,87.862817
6,Wednesday,634272.0,616213.5,18058.5,238507,556709.382678,2.847122,38.705254,87.771395
4,Thursday,677407.5,653925.0,23482.5,257214,597093.276249,3.466525,39.333869,88.143883
0,Friday,721254.0,698647.5,22606.5,255347,633838.523843,3.134333,36.548760,87.880071
2,Saturday,728715.0,702568.5,26146.5,279605,642342.228578,3.588028,39.797543,88.147249
3,Sunday,730203.0,705399.0,24804.0,249312,642788.596102,3.396864,35.343401,88.028753


In [15]:
# Export analysis tables for Tableau / reporting

outage_group.to_csv("data/outage_sla_2024.csv", index=False)
vendor_vol.to_csv("data/vendor_volume_2024.csv", index=True)
vendor_sat.to_csv("data/vendor_satisfaction_2024.csv", index=True)
vendor_sla.to_csv("data/vendor_sla_2024.csv", index=True)
missed_by_vendor.to_csv("data/missed_by_vendor_2024.csv", index=False)
dow_perf.to_csv("data/dow_performance_2024.csv", index=False)

print("Exported outage_sla_2024.csv, vendor_volume_2024.csv, vendor_satisfaction_2024.csv, vendor_sla_2024.csv, missed_by_vendor_2024.csv, dow_performance_2024.csv")


Exported outage_sla_2024.csv, vendor_volume_2024.csv, vendor_satisfaction_2024.csv, vendor_sla_2024.csv, missed_by_vendor_2024.csv, dow_performance_2024.csv
